In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
def temperature_extraction(path:str,years:list)-> pd.DataFrame:
    
    dataframe_list = []

    for year in years:
        print("-------------------------------------------------------------------------------------------")
        print(f'Reading data from year {year}')
        print("-------------------------------------------------------------------------------------------")
        temperature_path = os.path.join(path,str(year))

        temperature_files = [file for file in os.listdir(temperature_path) if os.path.isfile(os.path.join(temperature_path,file))]
        print(f"A Total of {len(temperature_files)} files read | Initiating file extraction")
        print("-------------------------------------------------------------------------------------------")
        n=1
        for file in temperature_files:
            print(f"Extracting file {n} from a total of {len(temperature_files)} | Progress: {np.round(n/len(temperature_files)*100,2)}%       ", end = "\r")
            file_path = os.path.join(temperature_path,file)
            extraction_df = pd.read_csv(file_path, 
                                        header = 8,
                                        sep = ';',
                                        encoding = 'latin-1')
            

            extraction_df['datetime'] = extraction_df['Data'] + " " + extraction_df['Hora UTC']

            extraction_df['datetime'] = pd.to_datetime(extraction_df['datetime'], format = '%Y/%m/%d %H%M %Z')
            
            str_file_list = file.split("_")

            region = str_file_list[1]
            state = str_file_list[2]
            city = str_file_list[4]

            extraction_df = extraction_df[["datetime","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"]]

            extraction_df['region'] = region
            extraction_df['state'] = state
            extraction_df['city'] = city

            extraction_df = extraction_df.rename(columns={"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temperature"})

            dataframe_list.append(extraction_df)
            n+=1
    df_raw = pd.concat(dataframe_list)

    return(df_raw)


In [3]:
def temperature_processing(df_raw:pd.DataFrame)->pd.DataFrame:

    df_raw['temperature'] = df_raw['temperature'].replace(",",".", regex = True)
    df_raw['temperature'] = df_raw['temperature'].astype(float)
    
    df_raw = df_raw[['datetime','temperature']]
    df_grouped = df_raw.groupby('datetime').mean().reset_index()
    
    return(df_grouped)
    

In [17]:
def load_extraction(path:str)-> pd.DataFrame:

    df_list = []
    print("-------------------------------------------------------------------------------------------")
    print(f'Reading data')
    print("-------------------------------------------------------------------------------------------")
    

    load_files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path,file))]
    print(f"A Total of {len(load_files)} files read | Initiating file extraction")
    print("-------------------------------------------------------------------------------------------")
    n=1
    for file in load_files:
        print(f"Extracting file {n} from a total of {len(load_files)} | Progress: {np.round(n/len(load_files)*100,2)}%       ", end = "\r")
        

        file_path = os.path.join(path,file)
        df_extraction = pd.read_csv(file_path, sep = ";")

        df_list.append(df_extraction)
        n+=1
    df_raw = pd.concat(df_list)

    return(df_raw)

In [13]:
def load_processing(df_raw:pd.DataFrame)-> pd.DataFrame:

    df_raw = df_raw.rename(columns = {'din_instante': 'datetime',
                          'nom_subsistema':'sub',
                          'val_cargaenergiahomwmed':'carga_sub',
                          'id_subsistema':'id_sub'} )
    
    df_raw  = df_raw .pivot_table(index = ["datetime"], columns=["id_sub"], values = "carga_sub")
    df_raw.reset_index(inplace = True)
    
    df_raw['datetime'] = pd.to_datetime(df_raw['datetime'])

    
    #Summing all coluns to reach total load
    sub_markets =  df_raw.select_dtypes(include=['float64', 'int64']).columns.to_list()
    df_raw['total_load'] = df_raw[sub_markets].sum(axis = 1)
    df_raw.head()

    df_processed = df_raw.drop(columns = sub_markets, axis = 1)

    return(df_processed)

In [21]:
load_path = "/mnt/e/github/load-forecast/01.database/raw/load"
df_raw = load_extraction(path = load_path)
df_processed = load_processing(df_raw = df_raw)

df_processed.to_csv(f"/mnt/e/github/load-forecast/01.database/processed/load/{2021}_{2023}_load_processed.csv")

-------------------------------------------------------------------------------------------
Reading data
-------------------------------------------------------------------------------------------
A Total of 3 files read | Initiating file extraction
-------------------------------------------------------------------------------------------


In [4]:
path = "/mnt/e/github/load-forecast/01.database/raw/temperature"
df_list = []
year_list = [2022,2023]

df_raw = temperature_extraction(path = path, years = year_list)
df_grouped = temperature_processing(df_raw = df_raw)

df_grouped.to_csv(f"/mnt/e/github/load-forecast/01.database/processed/temperature/{year_list[0]}_{year_list[1]}_temperature_processed.csv")

---------------------------------
Reading data from year 2022
---------------------------------
Files Read | Initiating file extraction
---------------------------------
---------------------------------of 567 | Progress: 100.0%       
Reading data from year 2023
---------------------------------
Files Read | Initiating file extraction
---------------------------------
